In [1]:
import pandas as pd
import seaborn as sns

In [54]:
metagenomics = pd.read_csv('/Volumes/PGH-Backup/ibd_data/metadata/hmp2_metagenomics_metadata.csv')

rnaseq = pd.read_csv('/Volumes/PGH-Backup/ibd_data/metadata/hmp2_rnaseq_metadata.tsv', sep='\t')

In [30]:
print(rnaseq.shape) 

display(rnaseq.head())

(251, 489)


,Project,Participant.ID,site_sub_coll,data_type,week_num,date_of_receipt,interval_days,visit_num,Research.Project,PDO.Number,...,FecalCal.received.at.MGH.,Proteomics.received.at.LBNL.,Stool.Sample.ID...Tube.A...EtOH.,Sample.ID..Tube.B..No.preservative.,Tube.A.and.B.received.at.Broad.,stool_id,smoking.status,Number.years.smoked,Age.when.started.smoking,How.many.cigarettes.cigars.etc..do.you.smoke.per.day.
0,C3002CSC2_TX,C3002,C3002CSC2,host_transcriptomics,0.0,NaN,NaN,1,ibdmdb,NaN,...,No,No,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN
1,C3002CSC1_TX,C3002,C3002CSC1,host_transcriptomics,0.0,NaN,NaN,1,ibdmdb,NaN,...,No,No,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN
2,C3002CSC4_TX,C3002,C3002CSC4,host_transcriptomics,0.0,NaN,NaN,1,ibdmdb,NaN,...,No,No,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN
3,C3002CSC3_TX,C3002,C3002CSC3,host_transcriptomics,0.0,NaN,NaN,1,ibdmdb,NaN,...,No,No,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN
4,C3003CSC1_TX,C3003,C3003CSC1,host_transcriptomics,1.0,NaN,NaN,1,ibdmdb,NaN,...,No,No,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Filter df1 based on the Participant ID column in df2
metagenomics_filt = metagenomics[metagenomics['Participant ID'].isin(rnaseq['Participant.ID'])]

print(metagenomics_filt['Participant ID'].unique())

In [56]:
rnaseq = rnaseq.rename(columns={'Participant.ID': 'Participant ID'})

In [ ]:
merged_df = pd.merge(rnaseq, metagenomics_filt, on='Participant ID', how='left', suffixes=('_rnaseq', '_metagenomics'))

display(merged_df)

In [58]:
# Filter for the closest timepoint or earliest if none is before or at the RNA-seq timepoint
def find_closest_timepoint(group):
    # Filter for metagenomic samples that are before or at the RNA-seq timepoint
    valid_samples = group[group['week_num_metagenomics'] <= group['week_num_rnaseq'].iloc[0]]
    
    if not valid_samples.empty:
        # Find the sample with the closest (maximum) timepoint
        closest_sample = valid_samples.loc[valid_samples['week_num_metagenomics'].idxmax()]
    else:
        # If no valid sample exists, return the earliest metagenomic sample
        closest_sample = group.loc[group['week_num_metagenomics'].idxmin()]
    
    return pd.Series([closest_sample['External ID'], closest_sample['week_num_metagenomics']], 
                     index=['External ID', 'week_num_metagenomics'])

In [59]:
closest_samples = merged_df.groupby(['Participant ID', 'Project_rnaseq']).apply(find_closest_timepoint).reset_index()

/var/folders/sm/rgfxp6g94f18340p2rvqnsc00000gn/T/ipykernel_23392/4172944390.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  closest_samples = merged_df.groupby(['Participant ID', 'Project_rnaseq']).apply(find_closest_timepoint).reset_index()


In [60]:
rnaseq = rnaseq.rename(columns={'Project': 'Project_rnaseq'})

In [61]:
# Merge the closest samples back to the original RNA-seq DataFrame
final_df = pd.merge(rnaseq, closest_samples, on=['Participant ID', 'Project_rnaseq'], how='left')

In [62]:
display(final_df)

,Project_rnaseq,Participant ID,site_sub_coll,data_type,week_num,date_of_receipt,interval_days,visit_num,Research.Project,PDO.Number,...,Stool.Sample.ID...Tube.A...EtOH.,Sample.ID..Tube.B..No.preservative.,Tube.A.and.B.received.at.Broad.,stool_id,smoking.status,Number.years.smoked,Age.when.started.smoking,How.many.cigarettes.cigars.etc..do.you.smoke.per.day.,External ID,week_num_metagenomics
0,C3002CSC2_TX,C3002,C3002CSC2,host_transcriptomics,0.0,NaN,NaN,1,ibdmdb,NaN,...,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,CSM5FZ3T_P,0.0
1,C3002CSC1_TX,C3002,C3002CSC1,host_transcriptomics,0.0,NaN,NaN,1,ibdmdb,NaN,...,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,CSM5FZ3T_P,0.0
2,C3002CSC4_TX,C3002,C3002CSC4,host_transcriptomics,0.0,NaN,NaN,1,ibdmdb,NaN,...,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,CSM5FZ3T_P,0.0
3,C3002CSC3_TX,C3002,C3002CSC3,host_transcriptomics,0.0,NaN,NaN,1,ibdmdb,NaN,...,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,CSM5FZ3T_P,0.0
4,C3003CSC1_TX,C3003,C3003CSC1,host_transcriptomics,1.0,NaN,NaN,1,ibdmdb,NaN,...,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,CSM5FZ4E_P,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,P6025CSC2_TX,P6025,P6025CSC2,host_transcriptomics,0.0,NaN,NaN,1,ibdmdb,NaN,...,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,PSM7J1BV,2.0
247,P6025CSC1_TX,P6025,P6025CSC1,host_transcriptomics,0.0,NaN,NaN,1,ibdmdb,NaN,...,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,PSM7J1BV,2.0
248,P6028CSC2_TX,P6028,P6028CSC2,host_transcriptomics,2.0,NaN,NaN,1,ibdmdb,NaN,...,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,PSM7J177,2.0
249,P6028CSC1_TX,P6028,P6028CSC1,host_transcriptomics,2.0,NaN,NaN,1,ibdmdb,NaN,...,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,PSM7J177,2.0


In [63]:
for_viewing = final_df[['Participant ID', 'External ID', 'week_num', 'week_num_metagenomics', 'Project_rnaseq']]

display(for_viewing)

,Participant ID,External ID,week_num,week_num_metagenomics,Project_rnaseq
0,C3002,CSM5FZ3T_P,0.0,0.0,C3002CSC2_TX
1,C3002,CSM5FZ3T_P,0.0,0.0,C3002CSC1_TX
2,C3002,CSM5FZ3T_P,0.0,0.0,C3002CSC4_TX
3,C3002,CSM5FZ3T_P,0.0,0.0,C3002CSC3_TX
4,C3003,CSM5FZ4E_P,1.0,2.0,C3003CSC1_TX
...,...,...,...,...,...
246,P6025,PSM7J1BV,0.0,2.0,P6025CSC2_TX
247,P6025,PSM7J1BV,0.0,2.0,P6025CSC1_TX
248,P6028,PSM7J177,2.0,2.0,P6028CSC2_TX
249,P6028,PSM7J177,2.0,2.0,P6028CSC1_TX


In [64]:
for_viewing.to_csv('/Volumes/PGH-Backup/ibd_data/metadata/matched_rnaseq_mgx_sample_mapping.tsv', index=False)